In [3]:
# !pip install catboost


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix,ConfusionMatrixDisplay,roc_curve,roc_auc_score,precision_recall_curve
from sklearn.ensemble import RandomForestClassifier , StackingClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score,GridSearchCV

# 한글 깨짐 방지
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'malgun Gothic'

In [5]:
test = pd.read_csv('../t-1/test.csv', encoding='euc-kr')
train = pd.read_csv('../t-1/train_Winsorization.csv', encoding='euc-kr')

In [6]:
train['기업수명주기'] = train['기업수명주기'].map({
    '도입기' : 1,
    '성장기' : 2,
    '성숙기' : 3,
    '수축기' : 4,
    '쇠퇴기' : 5
}).astype('category')

test['기업수명주기'] = test['기업수명주기'].map({
    '도입기' : 1,
    '성장기' : 2,
    '성숙기' : 3,
    '수축기' : 4,
    '쇠퇴기' : 5
}).astype('category')

train['파부비초과여부']=train['파부비초과여부'].astype('category')
test['파부비초과여부']=test['파부비초과여부'].astype('category')
train['파차의초과여부']=train['파차의초과여부'].astype('category')
test['파차의초과여부']=test['파차의초과여부'].astype('category')

In [7]:
# train=train[['총자본증가율',
# '총자산대비잉여현금흐름',
# '당좌자산회전률',
# '순운전자본회전률',
# '자기자본회전률',
# 'log자산총계',
# '자기자본증가율',
# '기업수명주기',
# '총자산대비현금흐름',
# '매출액대비잉여현금흐름',
# '영업이익증가율',
# '총자본투자효율',
# '총자본순이익률',
# '파부비초과여부',
# '파차의초과여부','t-1감사의견코드']]

# test=test[['총자본증가율',
# '총자산대비잉여현금흐름',
# '당좌자산회전률',
# '순운전자본회전률',
# '자기자본회전률',
# 'log자산총계',
# '자기자본증가율',
# '기업수명주기',
# '총자산대비현금흐름',
# '매출액대비잉여현금흐름',
# '영업이익증가율',
# '총자본투자효율',
# '총자본순이익률',
# '파부비초과여부',
# '파차의초과여부','t-1감사의견코드']]

In [8]:
X_train_int=train[['총자본증가율',
'총자산대비잉여현금흐름',
'당좌자산회전률',
'순운전자본회전률',
'자기자본회전률',
'log자산총계',
'자기자본증가율',
'총자산대비현금흐름',
'매출액대비잉여현금흐름',
'영업이익증가율',
'총자본투자효율',
'총자본순이익률',
't-1감사의견코드']]
X_train_cat=train[['기업수명주기','파부비초과여부','파차의초과여부']]

X_test_int=test[['총자본증가율',
'총자산대비잉여현금흐름',
'당좌자산회전률',
'순운전자본회전률',
'자기자본회전률',
'log자산총계',
'자기자본증가율',
'총자산대비현금흐름',
'매출액대비잉여현금흐름',
'영업이익증가율',
'총자본투자효율',
'총자본순이익률',
't-1감사의견코드']]
X_test_cat=test[['기업수명주기','파부비초과여부','파차의초과여부']]

In [9]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [10]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train_int)
X_test_sc = scaler.transform(X_test_int)

In [11]:
X_train_sc=pd.DataFrame(X_train_sc, columns=X_train_int.columns).drop('t-1감사의견코드',axis=1)
X_test_sc=pd.DataFrame(X_test_sc, columns=X_test_int.columns).drop('t-1감사의견코드',axis=1)

In [12]:
X_train_sum = pd.concat([X_train_sc,X_train_cat],axis=1)
X_test_sum = pd.concat([X_test_sc,X_test_cat],axis=1)

---

In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from imblearn.under_sampling import OneSidedSelection

def model_with_one_side_selection(x_train, y_train, x_test, y_test):
    categorical_features_indices = [12, 13, 14]

    models = [
        LogisticRegression(C=0.01, penalty='l2', random_state=0),
        RandomForestClassifier(random_state=0),
        LGBMClassifier(random_state=0),
        CatBoostClassifier(random_state=0, cat_features=categorical_features_indices, verbose=False),
        SVC(random_state=0)
    ]

    rdict = {
        'model': [],
        'sampling_ratio': [],
        'acc_train': [],
        'auc_train': [],
        'acc_test': [],
        'precision': [],
        'recall': [],
        'f1_score': [],
        'AUC_test': []
    }

    # One Side Selection을 사용하여 언더샘플링된 데이터 생성
    oss = OneSidedSelection()
    x_train_undersampled, y_train_undersampled = oss.fit_resample(x_train, y_train)

    for clf in models:
        # Train
        clf = clf.fit(x_train_undersampled, y_train_undersampled)
        y_hat_train = clf.predict(x_train_undersampled)
        results_train = (round(accuracy_score(y_train_undersampled, y_hat_train), 4), round(roc_auc_score(y_train_undersampled, y_hat_train), 4))

        # Test
        y_hat_test = clf.predict(x_test)
        results = (round(accuracy_score(y_test, y_hat_test), 4),
                   round(precision_score(y_test, y_hat_test), 4),
                   round(recall_score(y_test, y_hat_test), 4),
                   round(f1_score(y_test, y_hat_test), 4),
                   round(roc_auc_score(y_test, y_hat_test), 4))

        rdict['model'].append(type(clf).__name__)
        rdict['sampling_ratio'].append('One Side Selection')
        rdict['acc_train'].append(results_train[0])
        rdict['auc_train'].append(results_train[1])
        rdict['acc_test'].append(results[0])
        rdict['precision'].append(results[1])
        rdict['recall'].append(results[2])
        rdict['f1_score'].append(results[3])
        rdict['AUC_test'].append(results[4])

        confusion = confusion_matrix(y_test, y_hat_test)
        print(f"Model: {type(clf).__name__}, Sampling Ratio: One Side Selection")
        print(confusion)

    rdf_final = pd.DataFrame(data=rdict)
    return rdf_final

In [15]:
model_with_one_side_selection(X_train_sum, y_train, X_test_sum, y_test)

Model: LogisticRegression, Sampling Ratio: One Side Selection
[[37678   579]
 [ 2495   373]]
Model: RandomForestClassifier, Sampling Ratio: One Side Selection
[[37358   899]
 [ 1467  1401]]
Model: LGBMClassifier, Sampling Ratio: One Side Selection
[[37234  1023]
 [ 1358  1510]]
Model: CatBoostClassifier, Sampling Ratio: One Side Selection
[[37264   993]
 [ 1363  1505]]
Model: SVC, Sampling Ratio: One Side Selection
[[37790   467]
 [ 2127   741]]


,model,sampling_ratio,acc_train,auc_train,acc_test,precision,recall,f1_score,AUC_test
0,LogisticRegression,One Side Selection,0.9277,0.5576,0.9253,0.3918,0.1301,0.1953,0.5575
1,RandomForestClassifier,One Side Selection,1.0000,0.9997,0.9425,0.6091,0.4885,0.5422,0.7325
2,LGBMClassifier,One Side Selection,0.9516,0.7733,0.9421,0.5961,0.5265,0.5592,0.7499
3,CatBoostClassifier,One Side Selection,0.9621,0.8148,0.9427,0.6025,0.5248,0.5609,0.7494
4,SVC,One Side Selection,0.9401,0.6383,0.9369,0.6134,0.2584,0.3636,0.6231
